In [1]:
import pandas as pd
lines = pd.read_csv('spa.txt', names = ['src','tar','cc'], sep = '\t')
len(lines)

123770

In [2]:
#from sklearn.utils import shuffle
#lines = shuffle(lines)

In [3]:
lines = lines.drop(['cc'], axis = 1)
lines = lines[50000:100000]
lines.sample(10)

,src,tar
79485,I have to fix the washing machine.,Tengo que arreglar la lavadora.
52611,Tom climbed over the fence.,Tom trepó por encima de la cerca.
88172,You can put your name on a name tag.,Puedes poner tu nombre en una etiqueta.
64400,Japanese is our mother tongue.,El japonés es nuestra lengua materna.
75877,I never doubted you for a moment.,Nunca dudé de ti ni por un momento.
80311,My apartment is not far from here.,Mi departamento no queda lejos de aquí.
76232,I'm getting used to this weather.,Me estoy acostumbrando a este clima.
93074,"Tom didn't seem too surprised, either.","Tom no parecía demasiado soprendido, tampoco."
56934,Tom always speaks in French.,Tom siempre habla francés.
66295,Why is everyone looking at me?,¿Por qué me mira todo el mundo?


In [4]:
lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n')
lines.sample(10)

,src,tar
87437,This is a good learning environment.,\t Este es un buen entorno de aprendizaje. \n
71902,I just wanted to make you proud.,\t Solo quería enorgullecerlos. \n
82236,Do you like to live in the country?,\t ¿Le gusta vivir en el campo? \n
54834,I don't know either of them.,\t No conozco a ninguno de los dos. \n
60165,I've read all kinds of books.,\t He leído toda clase de libros. \n
79798,I want to talk privately with Tom.,\t Quiero hablar con Tom en privado. \n
72941,Running is good for your health.,\t Correr es bueno para la salud. \n
58999,He's accustomed to traveling.,\t Él está acostumbrado a viajar. \n
72775,My aunt brought me some flowers.,\t Mi tía me trajo algunas flores. \n
92532,She baked bread and cakes in the oven.,\t Ella cocinó pan y bizcochuelos en el horno. \n


In [5]:
src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [6]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

84
99


In [7]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '°']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [8]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'X': 46, 'Y': 47, 'Z': 48, 'a': 49, 'b': 50, 'c': 51, 'd': 52, 'e': 53, 'f': 54, 'g': 55, 'h': 56, 'i': 57, 'j': 58, 'k': 59, 'l': 60, 'm': 61, 'n': 62, 'o': 63, 'p': 64, 'q': 65, 'r': 66, 's': 67, 't': 68, 'u': 69, 'v': 70, 'w': 71, 'x': 72, 'y': 73, 'z': 74, '°': 75, 'á': 76, 'ã': 77, 'è': 78, 'é': 79, 'ö': 80, '‘': 81, '’': 82, '₂': 83}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, "'": 8, '(': 9, ')': 10, ',': 11, '-': 12, '.': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 3

In [9]:
encoder_input = []
for line in lines.src:
    temp_X = []
    for w in line:
        temp_X.append(src_to_index[w])
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[28, 57, 60, 60, 1, 63, 69, 68, 1, 68, 56, 57, 67, 1, 54, 63, 66, 61, 7, 1, 64, 60, 53, 49, 67, 53, 9], [28, 57, 60, 60, 1, 68, 56, 53, 1, 50, 69, 51, 59, 53, 68, 1, 71, 57, 68, 56, 1, 71, 49, 68, 53, 66, 9], [28, 57, 62, 49, 60, 60, 73, 7, 1, 31, 1, 56, 49, 70, 53, 1, 61, 73, 1, 63, 71, 62, 1, 51, 49, 66, 9], [28, 57, 62, 49, 60, 60, 73, 7, 1, 31, 1, 56, 49, 70, 53, 1, 61, 73, 1, 63, 71, 62, 1, 51, 49, 66, 9], [28, 57, 66, 67, 68, 7, 1, 60, 53, 68, 1, 61, 53, 1, 49, 67, 59, 1, 73, 63, 69, 1, 68, 56, 57, 67, 9]]


In [10]:
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
        temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 44, 57, 64, 64, 57, 66, 57, 3, 57, 71, 72, 57, 3, 58, 67, 70, 65, 73, 64, 53, 70, 61, 67, 11, 3, 68, 67, 70, 3, 58, 53, 74, 67, 70, 13, 3, 2], [1, 3, 38, 64, 57, 66, 53, 3, 57, 64, 3, 54, 53, 64, 56, 57, 3, 55, 67, 66, 3, 53, 59, 73, 53, 13, 3, 2], [1, 3, 32, 61, 66, 53, 64, 65, 57, 66, 72, 57, 3, 72, 57, 66, 59, 67, 3, 65, 61, 3, 68, 70, 67, 68, 61, 67, 3, 53, 73, 72, 67, 13, 3, 2], [1, 3, 32, 61, 66, 53, 64, 65, 57, 66, 72, 57, 3, 72, 57, 66, 59, 67, 3, 65, 61, 3, 68, 70, 67, 68, 61, 67, 3, 55, 67, 55, 60, 57, 13, 3, 2], [1, 3, 42, 70, 61, 65, 57, 70, 67, 3, 56, 89, 62, 53, 65, 57, 3, 68, 70, 57, 59, 73, 66, 72, 53, 70, 72, 57, 3, 57, 71, 72, 67, 13, 3, 2]]


In [11]:
decoder_target = []
for line in lines.tar:
    t = 0
    temp_X = []
    for w in line:
        if t > 0 :
            temp_X.append(tar_to_index[w])
        t = t + 1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 44, 57, 64, 64, 57, 66, 57, 3, 57, 71, 72, 57, 3, 58, 67, 70, 65, 73, 64, 53, 70, 61, 67, 11, 3, 68, 67, 70, 3, 58, 53, 74, 67, 70, 13, 3, 2], [3, 38, 64, 57, 66, 53, 3, 57, 64, 3, 54, 53, 64, 56, 57, 3, 55, 67, 66, 3, 53, 59, 73, 53, 13, 3, 2], [3, 32, 61, 66, 53, 64, 65, 57, 66, 72, 57, 3, 72, 57, 66, 59, 67, 3, 65, 61, 3, 68, 70, 67, 68, 61, 67, 3, 53, 73, 72, 67, 13, 3, 2], [3, 32, 61, 66, 53, 64, 65, 57, 66, 72, 57, 3, 72, 57, 66, 59, 67, 3, 65, 61, 3, 68, 70, 67, 68, 61, 67, 3, 55, 67, 55, 60, 57, 13, 3, 2], [3, 42, 70, 61, 65, 57, 70, 67, 3, 56, 89, 62, 53, 65, 57, 3, 68, 70, 57, 59, 73, 66, 72, 53, 70, 72, 57, 3, 57, 71, 72, 67, 13, 3, 2]]


In [12]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

41
79


In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target= pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [14]:
from tensorflow.keras.utils import to_categorical
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [15]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

W0518 21:07:04.329751 15308 deprecation.py:506] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _=decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [17]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=20, validation_split=0.2)

W0518 21:07:04.792513 15308 deprecation.py:323] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 576s 14ms/sample - loss: 1.0937 - val_loss: 1.0106
Epoch 2/20
40000/40000 [==============================] - 590s 15ms/sample - loss: 0.7374 - val_loss: 0.8519
Epoch 3/20
40000/40000 [==============================] - 602s 15ms/sample - loss: 0.6357 - val_loss: 0.7657
Epoch 4/20
40000/40000 [==============================] - 590s 15ms/sample - loss: 0.5750 - val_loss: 0.7093
Epoch 5/20
40000/40000 [==============================] - 591s 15ms/sample - loss: 0.5348 - val_loss: 0.6686
Epoch 6/20
40000/40000 [==============================] - 597s 15ms/sample - loss: 0.5053 - val_loss: 0.6423
Epoch 7/20
40000/40000 [==============================] - 585s 15ms/sample - loss: 0.4827 - val_loss: 0.6243
Epoch 8/20
40000/40000 [==============================] - 1374s 34ms/sample - loss: 0.4646 - val_loss: 0.6101
Epoch 9/20
40000/40000 [==============================] - 603s 15ms/sample - 

In [25]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)

In [26]:
#이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, 
                                                 initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

In [27]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs = [decoder_inputs] + decoder_states_inputs, 
                      outputs = [decoder_outputs] + decoder_states)

In [28]:
index_to_src = dict((i,char) for char, i in src_to_index.items())
index_to_tar = dict((i,char) for char, i in tar_to_index.items())

In [29]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1,tar_vocab_size))
    target_seq[0,0,tar_to_index['\t']] = 1.
    
    stop_condition = False
    decoded_sentence = ""
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        decoded_sentence += sampled_char
        
        if(sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True
            
        target_seq = np.zeros((1,1,tar_vocab_size))
        target_seq[0,0,sampled_token_index] = 1.
        
        states_value = [h, c]
        
    return decoded_sentence

In [93]:
print('입력 문장: ', end='')
sent = [input()]
input_sent = []
for line in sent:
    for w in line:
        input_sent.append(src_to_index[w])
input_sent = pad_sequences([input_sent], maxlen=max_src_len, padding='post')
inputSent = to_categorical(input_sent, 84)
decoded_sent = decode_sequence(inputSent)

print('번역기가 번역한 문장:', decoded_sent[:len(decoded_sent)-1])

입력 문장: Tom has a problem.
번역기가 번역한 문장:  Tom tiene un poco de agua. 
